In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("ws10.ipynb")

In [ ]:
rng_seed = 70

In [ ]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy as sp
import pandas as pd
#below line allows matplotlib plots to appear in cell output
%matplotlib inline

# **Question 1**: Double Pendulum Dynamics with SciPy Integration

In this question, you'll use **SciPy's numerical integration** to simulate the complex dynamics of a double pendulum system. The double pendulum is famous for exhibiting **chaotic behavior** - small changes in initial conditions can lead to dramatically different trajectories.

## Background: The Double Pendulum

A double pendulum consists of two pendulums attached end-to-end. The first pendulum is attached to a fixed pivot, and the second pendulum is attached to the end of the first.

### System Parameters
- **Mass 1**: $m_1$ (mass of first pendulum bob)
- **Mass 2**: $m_2$ (mass of second pendulum bob)  
- **Length 1**: $L_1$ (length of first pendulum)
- **Length 2**: $L_2$ (length of second pendulum)
- **Angles**: $\theta_1$ (angle of first pendulum from vertical), $\theta_2$ (angle of second pendulum from vertical)

### Equations of Motion

The Lagrangian approach yields the following coupled second-order differential equations:

$$m_1 L_1^2 \ddot{\theta_1} + m_2 L_1^2 \ddot{\theta_1} + m_2 L_1 L_2 \cos(\theta_1 - \theta_2) \ddot{\theta_2} + m_2 L_1 L_2 \sin(\theta_1 - \theta_2) \dot{\theta_2}^2 + (m_1 + m_2) g L_1 \sin(\theta_1) = 0$$

$$m_2 L_2^2 \ddot{\theta_2} + m_2 L_1 L_2 \cos(\theta_1 - \theta_2) \ddot{\theta_1} - m_2 L_1 L_2 \sin(\theta_1 - \theta_2) \dot{\theta_1}^2 + m_2 g L_2 \sin(\theta_2) = 0$$

### Converting to First-Order System

SciPy's `solve_ivp` requires a **first-order system**. We convert the second-order system by defining:
- $y_0 = \theta_1$ (angle 1)
- $y_1 = \dot{\theta_1}$ (angular velocity 1)
- $y_2 = \theta_2$ (angle 2)  
- $y_3 = \dot{\theta_2}$ (angular velocity 2)

This gives us the first-order system where:
$$\frac{dy}{dt} = \begin{bmatrix} y_1 \\ \ddot{\theta_1} \\ y_3 \\ \ddot{\theta_2} \end{bmatrix}$$

where $\ddot{\theta_1}$ and $\ddot{\theta_2}$ are obtained by solving the coupled equations above:

**Solved Angular Accelerations:**

The system can be written in matrix form as:
$$\mathbf{M} \begin{bmatrix} \ddot{\theta_1} \\ \ddot{\theta_2} \end{bmatrix} = \begin{bmatrix} f_1 \\ f_2 \end{bmatrix}$$

where the **mass matrix** $\mathbf{M}$ and **force vector** $\mathbf{f}$ are:

$$\mathbf{M} = \begin{bmatrix} 
(m_1 + m_2) L_1 & m_2 L_2 \cos(\theta_1 - \theta_2) \\
m_2 L_1 \cos(\theta_1 - \theta_2) & m_2 L_2
\end{bmatrix}$$

$$\mathbf{f} = \begin{bmatrix} 
-m_2 L_2 \dot{\theta_2}^2 \sin(\theta_1 - \theta_2) - (m_1 + m_2) g \sin(\theta_1) \\
m_2 L_1 \dot{\theta_1}^2 \sin(\theta_1 - \theta_2) - m_2 g \sin(\theta_2)
\end{bmatrix}$$

The angular accelerations are obtained by solving: $\begin{bmatrix} \ddot{\theta_1} \\ \ddot{\theta_2} \end{bmatrix} = \mathbf{M}^{-1} \mathbf{f}$

### Matrix Inversion Formula

For a $2 \times 2$ matrix $\mathbf{A} = \begin{bmatrix} a & b \\ c & d \end{bmatrix}$, the **inverse** is:

$$\mathbf{A}^{-1} = \frac{1}{\det(\mathbf{A})} \begin{bmatrix} d & -b \\ -c & a \end{bmatrix} = \frac{1}{ad - bc} \begin{bmatrix} d & -b \\ -c & a \end{bmatrix}$$

**Important:** The determinant $\det(\mathbf{A}) = ad - bc$ must be non-zero for the matrix to be invertible. In numerical computation, check that $|\det(\mathbf{A})| > \epsilon$ for some small tolerance $\epsilon$ (e.g., $10^{-10}$) to avoid division by near-zero values.

### Using `scipy.integrate.solve_ivp`

The `solve_ivp` function numerically integrates initial value problems of the form:
$$\frac{dy}{dt} = f(t, y), \quad y(t_0) = y_0$$

**Key parameters:**
- `fun`: Function defining the ODE system, `f(t, y)`
- `t_span`: Integration interval `[t_start, t_end]`
- `y0`: Initial conditions array
- `t_eval`: Specific times where you want the solution
- `method`: Integration method (default 'RK45' is usually good)

**Returns a solution object with:**
- `sol.t`: Time points
- `sol.y`: Solution array where `sol.y[i, j]` is the i-th component at time `sol.t[j]`

## **Part A**: Basic Double Pendulum Simulation

Implement `simulate_double_pendulum(theta1_0, theta2_0, omega1_0, omega2_0, t_max, N)` that:
1. Sets up the double pendulum equations as a first-order system
2. Uses `scipy.integrate.solve_ivp` to integrate the system
3. Returns time array and the two angle trajectories

**System Parameters** (use these fixed values):
- $m_1 = m_2 = 1$ kg
- $L_1 = L_2 = 1$ m  
- $g = 9.81$ m/s²

**Requirements:**
- Convert the coupled second-order equations to first-order system
- Use `solve_ivp` with time span `[0, t_max]` and `t_eval=np.linspace(0, t_max, N)`
- Within `solve_ivp` set `method='DOP853', rtol=1e-10, atol=1e-12` for high integration precision and to improve energy conservation
- Return `(t, theta1, theta2)` where each is a 1D numpy array

**Parameters:**
- `theta1_0`: float, initial angle of first pendulum (radians)
- `theta2_0`: float, initial angle of second pendulum (radians)  
- `omega1_0`: float, initial angular velocity of first pendulum (rad/s)
- `omega2_0`: float, initial angular velocity of second pendulum (rad/s)
- `t_max`: float, maximum simulation time (seconds)
- `N`: int, number of time points to evaluate

**Returns:**
- `t`: numpy array, time points
- `theta1`: numpy array, angle of first pendulum over time
- `theta2`: numpy array, angle of second pendulum over time

In [ ]:
def simulate_double_pendulum(theta1_0, theta2_0, omega1_0, omega2_0, t_max, N):
    from scipy.integrate import solve_ivp
    
    # System parameters
    m1, m2 = 1.0, 1.0  # masses (kg)
    L1, L2 = 1.0, 1.0  # lengths (m)
    g = 9.81  # gravity (m/s²)
    
    def double_pendulum_ode(t, y):
        \"\"\"
        Define the first-order ODE system for double pendulum.
        
        State vector: y = [theta1, omega1, theta2, omega2]
        \"\"\"
        # Extract state variables
        theta1, omega1, theta2, omega2 = y
        
        # Your code here: compute the angular accelerations using these expressions:

        # Check for singularity in mass matrix for inverting (denominator very close to zero)
        if abs(det_M) < 1e-10:
            # Handle near-singularity by using small regularization
            det_M = 1e-10 if det_M >= 0 else -1e-10
        
        return [omega1, alpha1, omega2, alpha2]
    
    # Set up initial conditions and time array
    # Use solve_ivp to integrate the system
    # Return t, theta1, theta2

In [ ]:
grader.check("q1a")

## **Part B**: Chaos Visualization - Multiple Trajectories

The double pendulum is a classic example of **deterministic chaos**. Even tiny differences in initial conditions can lead to dramatically different trajectories over time. This sensitivity to initial conditions is called the **"butterfly effect"**.

### Background: Phase Space and Trajectory Tracking

In this part, we'll visualize chaos by tracking the **tip of the second pendulum** as it moves through space. The position of the second pendulum's tip in Cartesian coordinates is:

$$x_2 = L_1 \sin(\theta_1) + L_2 \sin(\theta_2)$$
$$y_2 = -L_1 \cos(\theta_1) - L_2 \cos(\theta_2)$$

By plotting these $(x_2, y_2)$ coordinates over time for multiple slightly different initial conditions, we can see how small changes lead to completely different paths - the hallmark of chaotic behavior.

### Your Task

Implement `plot_chaos_trajectories(initial_conditions, t_max, N, show_plot=False)` that:
1. Simulates the double pendulum for multiple sets of initial conditions
2. Computes the tip position of the second pendulum for each trajectory  
3. Plots the trajectories in the x-y plane with different colors
4. Returns the figure object

**Requirements:**
- Use your `simulate_double_pendulum` function from Part A
- For each set of initial conditions, compute the (x2, y2) position of the second pendulum tip
- Plot each trajectory as a line with different colors
- Use the same system parameters: $L_1 = L_2 = 1$ m
- Plot configuration:
  - Figure size: `(12, 8)`
  - Equal aspect ratio: `ax.set_aspect('equal')`
  - Each trajectory: `linewidth=1.5`, `alpha=0.8`
  - X-axis label: "x₂ (m)"
  - Y-axis label: "y₂ (m)"  
  - Title: "Chaos in Double Pendulum: Tip Trajectories"
  - Grid with `alpha=0.3`
  - Legend showing "Trajectory 1", "Trajectory 2", etc.
- Only call `plt.show()` if `show_plot=True`
- Return the figure object

**Parameters:**
- `initial_conditions`: list of tuples, each tuple is `(theta1_0, theta2_0, omega1_0, omega2_0)`
- `t_max`: float, maximum simulation time (seconds)
- `N`: int, number of time points to evaluate
- `show_plot`: bool, default False. If True, display the plot

**Returns:**
- `fig`: matplotlib figure object

In [ ]:
def plot_chaos_trajectories(initial_conditions, t_max, N, show_plot=False):
    # System parameters
    L1, L2 = 1.0, 1.0  # lengths (m)
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Your code here:
    
    # Set equal aspect ratio
    
    # Set labels and title
    
    # Show plot if requested
    if show_plot:
        plt.show()
    
    return fig

In [ ]:
# Example: Demonstrate chaotic behavior with close initial conditions
print("Demonstrating Chaos: Tiny differences in initial conditions lead to vastly different trajectories")
print("=" * 80)

# Define very similar initial conditions - only tiny differences!
initial_conditions_chaos = [
    (np.pi/2, np.pi/2, 0.0, 0.0),        # Base case
    (np.pi/2 + 0.001, np.pi/2, 0.0, 0.0),      # Tiny change in theta1 (+0.001 rad ≈ 0.06°)
    (np.pi/2, np.pi/2 + 0.001, 0.0, 0.0),      # Tiny change in theta2 (+0.001 rad ≈ 0.06°)
    (np.pi/2, np.pi/2, 0.001, 0.0),      # Tiny change in omega1 (+0.001 rad/s)
    (np.pi/2, np.pi/2, 0.0, 0.001),      # Tiny change in omega2 (+0.001 rad/s)
]

print(f"Initial conditions (differences are only 0.001 radians or 0.001 rad/s):")
for i, ic in enumerate(initial_conditions_chaos):
    print(f"  Trajectory {i+1}: θ₁={ic[0]:.3f}, θ₂={ic[1]:.3f}, ω₁={ic[2]:.3f}, ω₂={ic[3]:.3f}")

print("\nSimulating for 15 seconds to show how small differences grow exponentially...")
fig_chaos = plot_chaos_trajectories(initial_conditions_chaos, t_max=15.0, N=2000, show_plot=True)

print("\nObservation: Despite nearly identical starting conditions, the trajectories")
print("diverge dramatically over time - this is the essence of deterministic chaos!")
plt.close(fig_chaos)

In [ ]:
#this cell saves a pendulum animation as .mp4, which can be downloaded or perhaps viewed on JupyterLab. 
#it takes ~30 seconds to run with current settings, feel free to play around with the system!

def animate_double_pendulums(initial_conditions, t_max, N, save_as=None):
    """
    Create an animated visualization of multiple double pendulums.
    
    Parameters:
    -----------
    initial_conditions : list of tuples
        Each tuple is (theta1_0, theta2_0, omega1_0, omega2_0)
    t_max : float
        Maximum simulation time (seconds) - animation duration will match this
    N : int
        Number of time points to evaluate
    save_as : str, optional
        If provided, save animation as this filename (e.g., 'pendulum.gif' or 'pendulum.mp4')
    
    Returns:
    --------
    fig : matplotlib figure object
    anim : matplotlib animation object
    """
    from matplotlib.animation import FuncAnimation
    
    # System parameters
    L1, L2 = 1.0, 1.0  # lengths (m)
    
    # Simulate all trajectories
    all_trajectories = []
    for theta1_0, theta2_0, omega1_0, omega2_0 in initial_conditions:
        t, theta1, theta2 = simulate_double_pendulum(theta1_0, theta2_0, omega1_0, omega2_0, t_max, N)
        all_trajectories.append((t, theta1, theta2))
    
    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.set_title('Double Pendulum Animation: Multiple Trajectories')
    
    # Colors for different pendulums
    colors = plt.cm.tab10(np.linspace(0, 1, len(initial_conditions)))
    
    # Initialize plot elements for each pendulum
    pendulum_lines = []
    pendulum_bobs = []
    trail_lines = []
    
    for i in range(len(initial_conditions)):
        # Pendulum bars (two lines: pivot to bob1, bob1 to bob2)
        line, = ax.plot([], [], 'o-', linewidth=3, markersize=8, color=colors[i], 
                       label=f'Pendulum {i+1}', alpha=0.8)
        pendulum_lines.append(line)
        
        # Trail of second bob
        trail, = ax.plot([], [], '-', linewidth=1, color=colors[i], alpha=0.3)
        trail_lines.append(trail)
    
    ax.legend(loc='upper right')
    
    # Store trail data
    trail_data = [{'x': [], 'y': []} for _ in range(len(initial_conditions))]
    
    def animate(frame):
        """Animation function called for each frame."""
        for i, (t, theta1, theta2) in enumerate(all_trajectories):
            # Get current angles
            if frame < len(theta1):
                th1, th2 = theta1[frame], theta2[frame]
                
                # Calculate positions
                x1 = L1 * np.sin(th1)
                y1 = -L1 * np.cos(th1)
                x2 = x1 + L2 * np.sin(th2)
                y2 = y1 - L2 * np.cos(th2)
                
                # Update pendulum bars (pivot -> bob1 -> bob2)
                pendulum_lines[i].set_data([0, x1, x2], [0, y1, y2])
                
                # Update trail
                trail_data[i]['x'].append(x2)
                trail_data[i]['y'].append(y2)
                
                # Limit trail length to last 100 points for performance
                if len(trail_data[i]['x']) > 100:
                    trail_data[i]['x'] = trail_data[i]['x'][-100:]
                    trail_data[i]['y'] = trail_data[i]['y'][-100:]
                
                trail_lines[i].set_data(trail_data[i]['x'], trail_data[i]['y'])
        
        return pendulum_lines + trail_lines
    
    # Create animation
    # Calculate frame rate to match t_max duration with max 30fps
    max_fps = 30
    desired_fps = min(max_fps, N / t_max)  # frames per second
    actual_interval = max(33, 1000 / desired_fps)  # milliseconds between frames (min 33ms for 30fps)
    
    anim = FuncAnimation(fig, animate, frames=N, interval=actual_interval, 
                        blit=True, repeat=True)
    
    # Save animation if filename provided
    if save_as:
        print(f"Saving animation as '{save_as}'...")
        print(f"Animation duration: {t_max:.1f} seconds at {desired_fps:.1f} fps")
        if save_as.endswith('.gif'):
            anim.save(save_as, writer='pillow', fps=desired_fps)
        elif save_as.endswith('.mp4'):
            anim.save(save_as, writer='ffmpeg', fps=desired_fps)
        else:
            # Default to GIF
            anim.save(save_as + '.gif', writer='pillow', fps=desired_fps)
        print(f"Animation saved successfully!")
    
    plt.tight_layout()
    plt.show()
    
    return fig, anim

# Example: Animate pendulums with slightly different initial conditions
print("Creating animation of double pendulums with close initial conditions...")
print("Watch how tiny differences lead to dramatically different motions!")

# Define initial conditions with small differences
initial_conditions_anim = [
    (np.pi/2, np.pi/2, 0.0, 0.0),      # Base case: both at 45 degrees (more stable)
    (np.pi/2 + 0.01, np.pi/2, 0.0, 0.0),  # Tiny change in first angle
    (np.pi/2, np.pi/2 + 0.01, 0.0, 0.0),  # Tiny change in second angle
]

print("Initial conditions:")
for i, ic in enumerate(initial_conditions_anim):
    print(f"  Pendulum {i+1}: θ₁={ic[0]:.3f}, θ₂={ic[1]:.3f}, ω₁={ic[2]:.3f}, ω₂={ic[3]:.3f}")

print("\nAnimating for 20 seconds...")
print("Note: Animation duration will match the t_max parameter (20 seconds)")
fig_anim, animation = animate_double_pendulums(initial_conditions_anim, t_max=15.0, N=1000, save_as='double_pendulum.mp4')

In [ ]:
grader.check("q1b")

## Required disclosure of use of AI technology

Please indicate whether you used AI to complete this homework. If you did, explain how you used it in the python cell below, as a comment.

In [ ]:
"""
# write ai disclosure here:

"""

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

Upload the .zip file to Gradescope!

In [ ]:
grader.export(pdf=False, force_save=True, run_tests=True)